Исходник датасета: https://www.kaggle.com/aerodinamicc/ecommerce-website-funnel-analysis


In [1]:
# Импортируем нужные бибилиотеки
import pandas as pd

In [2]:
# Загружаем в Python с помощью Pandas все датасеты
user_table = pd.read_csv('tables/user_table.csv')

# Просматриваем первые 5 строк датасета
user_table.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [3]:
user_table['year'] = pd.DatetimeIndex(user_table['date']).year
user_table['month'] = pd.to_datetime(user_table['date']).dt.month_name()
user_table['weekday'] = pd.to_datetime(user_table['date']).dt.day_name()

user_table.head()

,user_id,date,device,sex,year,month,weekday
0,450007,2015-02-28,Desktop,Female,2015,February,Saturday
1,756838,2015-01-13,Desktop,Male,2015,January,Tuesday
2,568983,2015-04-09,Desktop,Male,2015,April,Thursday
3,190794,2015-02-18,Desktop,Female,2015,February,Wednesday
4,537909,2015-01-15,Desktop,Male,2015,January,Thursday


In [4]:
# Просматриваем кол-во строк и столбцов каждого датасета
user_table.shape

(90400, 7)

In [5]:
home_page = pd.read_csv('tables/home_page_table.csv')
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [6]:
home_page['page'].unique()

array(['home_page'], dtype=object)

In [7]:
home_page

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page
...,...,...
90395,456851,home_page
90396,128619,home_page
90397,167290,home_page
90398,437765,home_page


In [8]:
home_page.shape

(90400, 2)

In [9]:
sum(home_page.duplicated())

0

In [10]:
search_page = pd.read_csv('tables/search_page_table.csv')
search_page.head()

,user_id,page
0,15866,search_page
1,347058,search_page
2,577020,search_page
3,780347,search_page
4,383739,search_page


In [11]:
search_page.shape

(45200, 2)

In [12]:
payment_page = pd.read_csv('tables/payment_page_table.csv')
payment_page.head()

,user_id,page
0,253019,payment_page
1,310478,payment_page
2,304081,payment_page
3,901286,payment_page
4,195052,payment_page


In [13]:
payment_page.shape

(6030, 2)

In [14]:
payment_confirmation = pd.read_csv('tables/payment_confirmation_table.csv')
payment_confirmation.head()

,user_id,page
0,123100,payment_confirmation_page
1,704999,payment_confirmation_page
2,407188,payment_confirmation_page
3,538348,payment_confirmation_page
4,841681,payment_confirmation_page


In [15]:
payment_confirmation.shape

(452, 2)

In [16]:
# Меняем название одинаковой колонки в каждом датасете
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation_page'})

In [17]:
# Склеиваем последовательно все датасеты с помощью метода merge по идентификатору user_id
df = user_table.merge(home_page, on = 'user_id', how = 'left')\
.merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left')\
.merge(payment_confirmation, on = 'user_id', how = 'left')

df.head()

,user_id,date,device,sex,year,month,weekday,home_page,search_page,payment_page,payment_confirmation_page
0,450007,2015-02-28,Desktop,Female,2015,February,Saturday,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,2015,January,Tuesday,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,2015,April,Thursday,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,2015,February,Wednesday,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,2015,January,Thursday,home_page,NaN,NaN,NaN


In [18]:
df.shape

(90400, 11)

In [19]:
# Второй способ: склеиваем все датасеты сразу
from functools import reduce

data_frames = [user_table, home_page, search_page, payment_page, payment_confirmation]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on='user_id', how='left'), data_frames)

df_merged.head()

,user_id,date,device,sex,year,month,weekday,home_page,search_page,payment_page,payment_confirmation_page
0,450007,2015-02-28,Desktop,Female,2015,February,Saturday,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,2015,January,Tuesday,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,2015,April,Thursday,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,2015,February,Wednesday,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,2015,January,Thursday,home_page,NaN,NaN,NaN


In [20]:
df_merged.shape

(90400, 11)

In [21]:
df['sex'].unique()

array(['Female', 'Male'], dtype=object)

In [22]:
df['home_page'] = df['home_page'].fillna(0)
condition = df['home_page'] == 'home_page'
df.loc[condition, 'home_page'] = 1
df.head()

df['search_page'] = df['search_page'].fillna(0)
condition = df['search_page'] == 'search_page'
df.loc[condition, 'search_page'] = 1
df.head()

df['payment_page'] = df['payment_page'].fillna(0)
condition = df['payment_page'] == 'payment_page'
df.loc[condition, 'payment_page'] = 1
df.head()

df['payment_confirmation_page'] = df['payment_confirmation_page'].fillna(0)
condition = df['payment_confirmation_page'] == 'payment_confirmation_page'
df.loc[condition, 'payment_confirmation_page'] = 1
df.head()

,user_id,date,device,sex,year,month,weekday,home_page,search_page,payment_page,payment_confirmation_page
0,450007,2015-02-28,Desktop,Female,2015,February,Saturday,1,0,0,0
1,756838,2015-01-13,Desktop,Male,2015,January,Tuesday,1,0,0,0
2,568983,2015-04-09,Desktop,Male,2015,April,Thursday,1,1,0,0
3,190794,2015-02-18,Desktop,Female,2015,February,Wednesday,1,1,0,0
4,537909,2015-01-15,Desktop,Male,2015,January,Thursday,1,0,0,0


In [23]:
df['sex'].value_counts()

sex
Male      45325
Female    45075
Name: count, dtype: int64

In [24]:
# Агрегируем данные: подсчитываем общее количество пользователей и сколько из них подтвердили покупку
df['payment_confirmation_page'].value_counts(normalize=True)

payment_confirmation_page
0    0.995
1    0.005
Name: proportion, dtype: float64

In [25]:
# Агрегируем данные: подсчитываем сколько пользователей совершило каждое из 4х событий на сайте
summary = df[['home_page', 'search_page', 'payment_page', 'payment_confirmation_page']].sum().reset_index()
summary.head()

,index,0
0,home_page,90400
1,search_page,45200
2,payment_page,6030
3,payment_confirmation_page,452


In [26]:
# Для удобства переименуем столбцы
summary = summary.rename(columns = {'index':'stage', 0:'number'})
summary.head()

,stage,number
0,home_page,90400
1,search_page,45200
2,payment_page,6030
3,payment_confirmation_page,452


In [27]:
# Импортируем графическую библиотеку Plotly (https://plotly.com/python/funnel-charts/)
from plotly import graph_objects as go
import plotly.io as pio

# Настраиваем скрипт этой библиотеки для построения воронки конверсий нашими параметрами
fig = go.Figure(go.Funnel(
    y = summary['stage'],
    x = summary['number'],
    textinfo = "value+percent previous"))

fig.update_layout(
    title = 'Воронка конверсии Интернет-магазина',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

pio.write_image(fig, 'plots/plot.png')

![Plot](plots/plot.png)

In [28]:
dfg = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation_page']].sum().unstack('sex').unstack('sex').reset_index()

dfg.columns.name = None

dfg = dfg.rename(columns = {'index':'Page'})

dfg.head()

,Page,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation_page,241,211


In [29]:
# Строим более продвинутый вариант воронки конверсий на сайте с сегментацией по полу
from plotly import graph_objects as go
import plotly.io as pio

# Настраиваем скрипт библиотеки Plotly нашими параметрами
fig = go.Figure()

# Часть графика для пола Male
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Male'],
    name = 'Male',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

# Часть графика для пола Female
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Female'],
    name = 'Female',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии по полу',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

pio.write_image(fig, 'plots/plot2.png')

![Plot](plots/plot2.png)

А теперь потренируемся в создании сводных таблиц методами groupby, pivot_table, crosstab для ответов на поставленные вопросы:

1) Какой месяц самый топовый по продажам?

In [30]:
# Используем метод pivot_table
pd.pivot_table(df, index = 'sex', columns = 'month', values = 'payment_confirmation_page', aggfunc = 'sum', margins=True)

month,April,February,January,March,All
sex,,,,,
Female,20,86,109,26,241
Male,26,87,80,18,211
All,46,173,189,44,452


2) Какой день недели самый топовый по продажам? Также есть ответ на вопрос, в какой день недели лучше покупают мужичины и женщины.

In [31]:
# Используем метод pivot_table
pd.pivot_table(df, index = 'sex', columns = 'weekday', values = 'payment_confirmation_page', aggfunc = 'sum', margins=True)

weekday,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,All
sex,,,,,,,,
Female,26,43,36,35,31,40,30,241
Male,28,34,34,27,35,29,24,211
All,54,77,70,62,66,69,54,452
